## DenseNet

 - Training Dataset:  RandAugment, N=1, M=20
   Validation with Unaugmented Data
 - Sagemaker Notebook must be of type, conda_pytorch_p36
 
#### Install Requirements

In [4]:
import boto3
import numpy 
import sagemaker
from sagemaker.pytorch import PyTorch
import torch
import os

In [5]:
!pip install -r '/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/requirements.txt'
!pip install tensorboard

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


#### Train the Model

In [ ]:
# Train the model per the settings specified in the original paper
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python train.py --config configs/cifar/densenet.yaml \
    model.densenet.depth 100 \
    model.densenet.growth_rate 12 \
    train.batch_size 64 \
    train.base_lr 0.1 \
    dataset.name CIFAR10_RA_1_20 \
    train.output_dir /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_1_20_c10val/exp00 \
    scheduler.epochs 400

[2020-07-19 11:10:55] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10_RA_1_20
  dataset_dir: ''
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: densenet
  init_mode: kaiming_fan_out
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_type: bottlenec

#### Refine the Model

In [ ]:
# Resume training with the un-augmented data
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python train.py --config /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_1_20_c10val/exp00/config.yaml \
    train.checkpoint /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_1_20_c10val/exp00/checkpoint_00400.pth \
    dataset.name CIFAR10 \
    train.base_lr .001 \
    train.output_dir /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_1_20_c10val/exp00_resume400_50 \
    scheduler.epochs 50


[2020-07-20 14:35:54] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10
  dataset_dir: ~/.torch/datasets/CIFAR10
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: densenet
  init_mode: kaiming_fan_out
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_

#### Evaluate Test Datasets

In [7]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Write the results to the test output directory specified.
!python evaluate.py --config configs/cifar/densenet.yaml \
   model.densenet.depth 100 \
   model.densenet.growth_rate 12 \
   test.batch_size 64 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_1_20_c10val/exp00_resume400_50/checkpoint_00050.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_1_20_c10val/exp00_resume400_50/test_results_0050_cifar10

[2020-07-20 23:02:57] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_1_20_c10val/exp00_resume400_50/checkpoint_00050.pth
Files already downloaded and verified
100%|█████████████████████████████████████████| 157/157 [00:19<00:00,  8.22it/s]
[2020-07-20 23:03:18] __main__ INFO: Elapsed 19.10
[2020-07-20 23:03:18] __main__ INFO: Loss 0.2574 Accuracy 0.9345


In [8]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Write the results to the test output directory specified.
!python evaluate.py --config configs/cifar/densenet.yaml \
   model.densenet.depth 100 \
   model.densenet.growth_rate 12 \
   test.batch_size 64 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_1_20_c10val/exp00/checkpoint_00400.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_1_20_c10val/exp00/test_results_0400_cifar10

[2020-07-20 23:03:51] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_1_20_c10val/exp00/checkpoint_00400.pth
Files already downloaded and verified
100%|█████████████████████████████████████████| 157/157 [00:18<00:00,  8.29it/s]
[2020-07-20 23:04:11] __main__ INFO: Elapsed 18.94
[2020-07-20 23:04:11] __main__ INFO: Loss 0.5418 Accuracy 0.8866


In [9]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# Write the results to the test output directory specified.
!python evaluate.py --config configs/cifar/densenet.yaml \
   model.densenet.depth 100 \
   model.densenet.growth_rate 12 \
   test.batch_size 64 \
   dataset.name CIFAR101 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_1_20_c10val/exp00_resume400_50/checkpoint_00050.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_1_20_c10val/exp00_resume400_50/test_results_0050_cifar101

[2020-07-20 23:04:29] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_1_20_c10val/exp00_resume400_50/checkpoint_00050.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 32/32 [00:04<00:00,  7.69it/s]
[2020-07-20 23:04:34] __main__ INFO: Elapsed 4.16
[2020-07-20 23:04:34] __main__ INFO: Loss 0.6161 Accuracy 0.8530


In [10]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# Write the results to the test output directory specified.
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python evaluate.py --config configs/cifar/densenet.yaml \
    model.densenet.depth 100 \
    model.densenet.growth_rate 12 \
    test.batch_size 64 \
    dataset.name CIFAR101 \
    test.checkpoint /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_1_20_c10val/exp00/checkpoint_00400.pth \
    test.output_dir /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_1_20_c10val/exp00/test_results_0400_cifar101

[2020-07-20 23:04:54] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_1_20_c10val/exp00/checkpoint_00400.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 32/32 [00:04<00:00,  7.96it/s]
[2020-07-20 23:04:59] __main__ INFO: Elapsed 4.02
[2020-07-20 23:04:59] __main__ INFO: Loss 1.0834 Accuracy 0.7875


#### Record the Results

In [11]:
import pandas as pd
model = 'densenet_BC_100_12_ra_1_20_c10val'
model_refined = model + '_refined400'

a = pd.Series([model, 400, 'cifar10', 0.5418, 0.8866])
c = pd.Series([model, 400, 'cifar10.1', 1.0834, 0.7875])

e = pd.Series([model_refined, 50, 'cifar10.1', 0.6161, 0.8530])
f = pd.Series([model_refined, 50, 'cifar10', 0.2574, 0.9345])
               
df_results = pd.concat([a,c,e,f], axis=1).T
df_results.columns = ['Model', 'Epoch', 'Testset', 'Loss', 'Accuracy']

df_results['Original_Accuracy'] = df_results.apply((lambda row: 95.5 if row[2] == 'cifar10' else 87.6), axis=1)
df_results['Original_CI'] = df_results.apply((lambda row: (95.1, 95.9) if row[2] == 'cifar10' else (86.1, 89.0)), axis=1)

df_results.to_csv('/home/ec2-user/SageMaker/experiments/' + model + '/results.csv')
df_results

,Model,Epoch,Testset,Loss,Accuracy,Original_Accuracy,Original_CI
0,densenet_BC_100_12_ra_1_20_c10val,400,cifar10,0.5418,0.8866,95.5,"(95.1, 95.9)"
1,densenet_BC_100_12_ra_1_20_c10val,400,cifar10.1,1.0834,0.7875,87.6,"(86.1, 89.0)"
2,densenet_BC_100_12_ra_1_20_c10val_refined400,50,cifar10.1,0.6161,0.853,87.6,"(86.1, 89.0)"
3,densenet_BC_100_12_ra_1_20_c10val_refined400,50,cifar10,0.2574,0.9345,95.5,"(95.1, 95.9)"


In [12]:
# Upload the model checkpoints, configs, and results to S3 
bucket='sagemaker-may29'
prefix = 'sagemaker/results/original-models/densenet_BC_100_12_ra_1_20_c10val'
path = '/home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_1_20_c10val'

s3_resource = boto3.resource("s3", region_name="us-east-2")

def uploadDirectory(local_path,bucket_name,s3_prefix):

    my_bucket = s3_resource.Bucket(bucket_name)
    
    for path, subdirs, files in os.walk(local_path):
        path = path.replace("\\","/")
        directory_name = path.replace(local_path,"")
        for file in files:
            #print("Local File:", os.path.join(path, file))
            #print("      Dest:", s3_prefix+directory_name+'/'+file)
            my_bucket.upload_file(os.path.join(path, file), s3_prefix+directory_name+'/'+file)
    
uploadDirectory(path,bucket,prefix)